In [ ]:
import os, sys, time, random
import h5py
import numpy as np

In [2]:
from keras.layers import *
from keras.models import *
from keras.utils import *
from keras.initializers import *
import tensorflow as tf

In [3]:
#training設定
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000

data_path = "tc.txt"

In [4]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [5]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, erase = line.split('\t')
    #用"\t"作為SOS
    #以"\n"作為EOS
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [6]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 73
Number of unique output tokens: 2037
Max sequence length for inputs: 26
Max sequence length for outputs: 22


In [7]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [8]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
#去掉encoders的輸出，只保留狀態h及c
encoder_states = [state_h, state_c]

#以encoders保留的h及c狀態進行初始化
decoder_inputs = Input(shape=(None, num_decoder_tokens))

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [9]:
model.summary()
print(model.layers[-3].output)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 73)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 2037)  0           []                               
                                ]                                                                 
                                                                                                  
 lstm (LSTM)                    [(None, 256),        337920      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                 

In [10]:
print("encoder_input_data shape:",encoder_input_data.shape)
print("decoder_input_data shape:",decoder_input_data.shape)
print("decoder_target_data shape:",decoder_target_data.shape)

encoder_input_data shape: (10000, 26, 73)
decoder_input_data shape: (10000, 22, 2037)
decoder_target_data shape: (10000, 22, 2037)


In [11]:
#training
from keras.optimizers import * 
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.001), loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
           batch_size=batch_size,
           epochs=epochs, 
           validation_split=0.2)

G:\anaconda20211117\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/100
125/125 [==============================] - 28s 207ms/step - loss: 1.8467 - val_loss: 2.1991
Epoch 2/100
125/125 [==============================] - 24s 196ms/step - loss: 1.7616 - val_loss: 2.1558
Epoch 3/100
125/125 [==============================] - 24s 190ms/step - loss: 1.7039 - val_loss: 2.0902
Epoch 4/100
125/125 [==============================] - 24s 194ms/step - loss: 1.6199 - val_loss: 2.0288
Epoch 5/100
125/125 [==============================] - 24s 194ms/step - loss: 1.5558 - val_loss: 1.9594
Epoch 6/100
125/125 [==============================] - 24s 192ms/step - loss: 1.4907 - val_loss: 1.9118
Epoch 7/100
125/125 [==============================] - 24s 192ms/step - loss: 1.4226 - val_loss: 1.8292
Epoch 8/100
125/125 [==============================] - 24s 196ms/step - loss: 1.3354 - val_loss: 1.7753
Epoch 9/100
125/125 [==============================] - 24s 190ms/step - loss: 1.2581 - val_loss: 1.6963
Epoch 10/100
125/125 [==============================] - 23s 186m

In [12]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [13]:
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #以\t當作SOS
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = [h, c]
    return decoded_sentence

In [16]:
for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('--------------------------------------')
    print('輸入:', input_texts[seq_index])
    print('輸出:', decoded_sentence)

--------------------------------------
輸入: Hi.
輸出: 嗨！

--------------------------------------
輸入: Hi.
輸出: 嗨！

--------------------------------------
輸入: Run.
輸出: 你走跑。

--------------------------------------
輸入: Stop!
輸出: 住手！

--------------------------------------
輸入: Wait!
輸出: 等一下！

--------------------------------------
輸入: Wait!
輸出: 等一下！

--------------------------------------
輸入: Begin.
輸出: 開始！

--------------------------------------
輸入: Hello!
輸出: 你好。

--------------------------------------
輸入: I try.
輸出: 我試試。

--------------------------------------
輸入: I won!
輸出: 我贏了。

--------------------------------------
輸入: Oh no!
輸出: 不會吧。

--------------------------------------
輸入: Cheers!
輸出: 乾杯!

--------------------------------------
輸入: Got it?
輸出: 你懂了嗎？

--------------------------------------
輸入: Got it?
輸出: 你懂了嗎？

--------------------------------------
輸入: Got it?
輸出: 你懂了嗎？

--------------------------------------
輸入: He ran.
輸出: 他跑了。

--------------------------------------
輸入: Hop in.


In [41]:
data_path2 = "test.txt"
input_texts2 = []
#target_texts2 = []
input_characters2 = set()
#target_characters2 = set()
with open(data_path2, 'r', encoding='utf-8') as f:
    lines2 = f.read().split('\n')

In [42]:
for line in lines2[: min(num_samples, len(lines2) - 1)]:
    input_text2, target_text2, erase = line.split('\t')
    input_texts2.append(input_text2)
    for char in input_text2:
        if char not in input_characters2:
            input_characters2.add(char)
input_characters2 = sorted(list(input_characters2))
num_encoder_tokens2 = len(input_characters2)
max_encoder_seq_length2 = max([len(txt) for txt in input_texts2])

In [43]:
input_token_index2 = dict(
    [(char, i) for i, char in enumerate(input_characters2)])
encoder_input_data2 = np.zeros(
    (len(input_texts2), max_encoder_seq_length2, num_encoder_tokens2),
    dtype='float32')
for i, (input_text, target_text) in enumerate(zip(input_texts2, target_texts2)):
    for t, char in enumerate(input_text):
        encoder_input_data2[i, t, input_token_index2[char]] = 1.

In [44]:
for seq_index in range(5):
    input_seq2 = encoder_input_data2[seq_index: seq_index + 1]
    decoded_sentence2 = decode_sequence(input_seq2)
    print('--------------------------------------')
    print('輸入:', input_texts2[seq_index])
    print('輸出:', decoded_sentence2)

--------------------------------------
輸入: I don't really think so.
輸出: 我沒什麼積積。

--------------------------------------
輸入: Are you sure about that?
輸出: 你還在那裡嗎？

--------------------------------------
輸入: I got fired from the company.
輸出: 我把它扔了嗎?

--------------------------------------
輸入: I can't remember how to go there.
輸出: 我聽到見到他們積了。

--------------------------------------
輸入: Wait!
輸出: 等等！

